In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Word Count") \
    .getOrCreate()
sc = spark.sparkContext

# 题目一

In [2]:
lines = sc.textFile('chapter4-data01.txt')
line=lines.distinct()

# 该系学生数目

In [3]:
res1 = line.map(lambda x:x.split(",")).map(lambda x:x[0])
distinct_res1 = res1.distinct()
distinct_res1.count()

265

# 该系总共开设的课程

In [4]:
res2 = line.map(lambda x:x.split(",")).map(lambda x:x[1])
distinct_res2 = res2.distinct()
distinct_res2.count()

8

# 计算TOM同学的总平均分

In [5]:
res3 = line.map(lambda x:x.split(",")).filter(lambda x:x[0]=="Tom")
res3.foreach(print)
score = res3.map(lambda x:int(x[2]))
num = res3.count()
sum_score = score.reduce(lambda x,y:x+y)
avg = sum_score/num
print(avg)

30.8


# 每名同学的选修了几门科目

In [6]:
print(line.map(lambda x:(x.split(",")[0],1)).reduceByKey(lambda x,y:x+y).collect())

[('Aaron', 4), ('Adair', 3), ('Adonis', 5), ('Alan', 5), ('Aldrich', 3), ('Alexander', 4), ('Alvin', 5), ('Amos', 5), ('Andrew', 4), ('Antonio', 3), ('Archer', 5), ('Armstrong', 2), ('Baron', 6), ('Barry', 5), ('Bartholomew', 5), ('Bart', 5), ('Beck', 4), ('Ben', 4), ('Benedict', 6), ('Benjamin', 4), ('Bennett', 6), ('Benson', 4), ('Bernie', 3), ('Bertram', 3), ('Bill', 2), ('Bishop', 2), ('Blithe', 3), ('Booth', 6), ('Borg', 4), ('Boris', 6), ('Boyd', 3), ('Brian', 6), ('Brook', 4), ('Bruno', 5), ('Channing', 4), ('Christian', 2), ('Clarence', 7), ('Clark', 6), ('Claude', 2), ('Clement', 5), ('Cleveland', 4), ('Cliff', 5), ('Colbert', 4), ('Colby', 4), ('Colin', 5), ('Conrad', 2), ('Corey', 4), ('Dean', 7), ('Devin', 4), ('Don', 2), ('Donahue', 5), ('Duke', 4), ('Duncann', 5), ('Edward', 4), ('Elijah', 4), ('Ellis', 4), ('Elmer', 4), ('Elroy', 5), ('Elton', 5), ('Enoch', 3), ('Francis', 4), ('Geoffrey', 4), ('Giles', 7), ('Gordon', 4), ('Greg', 4), ('Harlan', 6), ('Hayden', 3), ('Hila

# 该系课程DataBase课程有多少人选修

In [7]:
res4 = line.map(lambda x:x.split(",")).filter(lambda x:x[1]=='DataBase')
res4.count()

126

# 每门科目的平均分是多少

In [8]:
res5 = line.map(lambda x:x.split(",")).map(lambda x:(x[1],(int(x[2]),1)))
temp2 = res5.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))
avg2 = temp2.map(lambda x:(x[0],round(x[1][0]/x[1][1],2)))
df_avg2 = spark.createDataFrame(avg2)
df_avg2.show()

+---------------+-----+
|             _1|   _2|
+---------------+-----+
|ComputerNetwork| 51.9|
|         Python|57.82|
|  DataStructure|47.57|
|OperatingSystem|54.94|
|       DataBase|50.54|
|      Algorithm|48.83|
|      CLanguage|50.61|
|       Software|50.91|
+---------------+-----+



# 使用累加器计算共有多少人选了DataBase这门课

In [15]:
res6 = line.map(lambda x:x.split(",")).filter(lambda x:x[1]=="DataBase")
accum = sc.accumulator(0)
res.foreach(lambda x:accum.add(1))
accum.value

1073